In [1]:
#generate dataset
import gym
import numpy as np
from stable_baselines import SAC

#we should get normal transitions, and add noise as samples
#pendulum obs: cos(theta), sin(theta), thetadot, action: 1

#config page
total_episodes = 10000
ac_dim = 1
ob_dim = 3
x_dim = 2*ob_dim+ac_dim
#config page end

env = gym.make('NoisyPendulum-v0')
model=SAC.load("sac_pendulum")
obs = env.reset()
acvariant = np.random.rand(total_episodes)
cor_dataset_xraw = np.zeros((total_episodes,x_dim))
cor_dataset_y = np.zeros((total_episodes,ob_dim))

subcount = 0
for dd in range(total_episodes):
    action, states = model.predict(obs)
    action = action + acvariant[dd]
    action = np.clip(action, -2, 2)[0]
    cor_dataset_xraw[dd][0:ob_dim] = obs
    cor_dataset_xraw[dd][ob_dim:ob_dim+ac_dim] = action
    obs, rewards, dones, info = env.step(action)
    cor_dataset_xraw[dd][ob_dim+ac_dim:2*ob_dim+ac_dim] = obs
    cor_dataset_y[dd][:] = obs

    subcount = subcount + 1
    if subcount >= 100:
        subcount = 0
        obs = env.reset()

np.save("cor_x_raw.npy",cor_dataset_xraw)
np.save("cor_y.npy",cor_dataset_y)

from winsound import Beep
Beep(3000, 500)

In [4]:
#add random noises 
import numpy as np
noiseangle = np.random.rand(100)-0.5
noisespeed = 0.5*np.random.rand(100)-0.25
np.save("cor_x_raw.npy",Xraw)
np.save("cor_y.npy",Y)
total_episodes = 10000
ac_dim = 1
ob_dim = 3
x_dim = 2*ob_dim+ac_dim
X = np.zeros((total_episodes,x_dim))

for outer in range(100):
    cosb = np.cos(noiseangle[outer])
    sinb = np.sin(noiseangle[outer])
    bias = noisespeed[outer]
    for inner in range(100):
        count = outer*100 + inner
        X[count][0] = Xraw[count][0]*cosb - Xraw[count][1]*sinb
        X[count][1] = Xraw[count][1]*cosb + Xraw[count][0]*sinb
        X[count][2] = Xraw[count][2] + bias
        X[count][4] = Xraw[count][4]*cosb - Xraw[count][5]*sinb
        X[count][5] = Xraw[count][5]*cosb + Xraw[count][4]*sinb
        X[count][6] = Xraw[count][6] + bias


[0.29011866 0.30202231 0.08115128 0.08273387 0.33780802 0.35604105
 0.28712655 0.3054427  0.08136879 0.09203884 0.22069784 0.22646592
 0.45178728 0.02004057 0.14435043 0.07626262 0.2306091  0.27277655
 0.4033471  0.11246322 0.49691709 0.05774639 0.38139788 0.4417823
 0.09899006 0.49473791 0.1472519  0.1196457  0.28019263 0.38325269
 0.2681148  0.43088985 0.45026283 0.20496248 0.42723858 0.31070979
 0.09765718 0.16168876 0.45054174 0.06373099 0.09380421 0.27843196
 0.40883514 0.01065791 0.22239993 0.32502354 0.23448681 0.0283417
 0.41095463 0.07390841 0.21465365 0.30442996 0.32309391 0.47874008
 0.17337088 0.3584716  0.47039838 0.23872258 0.30166996 0.49575689
 0.18991886 0.32364925 0.08214712 0.47008255 0.39862618 0.33841472
 0.21352536 0.07834937 0.35663275 0.02215653 0.01021925 0.03143671
 0.19904639 0.04565591 0.38186042 0.15023564 0.01364519 0.34051459
 0.20917306 0.44220204 0.47515315 0.02917297 0.44248576 0.44078769
 0.03314344 0.33620358 0.38348137 0.04918304 0.18655418 0.264490

In [4]:
#train neural network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

input_size = x_dim
output_size = ob_dim
hidden_size = [32,32]

model = Sequential()
model.add(Dense(hidden_size[0], input_dim = input_size, init = 'uniform', activation = 'relu')
)
model.add(Dense(hidden_size[1], init = 'uniform', activation = 'relu')
)
model.add(Dense(output_size),init = 'uniform', activation = None)

model.compile(loss='mse', optimizer='adam', metrics=['mse'])

model.fit(x_train,y_train,epochs = 100, batch_size = 64)

y_pred = model.predict(x_val)

[list([1, 2]) list([3, 4, 5])]


In [ ]:
#test performance
import gym
import numpy as np

from stable_baselines import SAC

env = gym.make('NoisyPendulum-v0')
dcs_model=SAC.load("sac_pendulum")
obs_fix = env.reset()
obs_raw = obs_fix
env.set_noise(0,0.18)
sub_reward_fix=[]
for dd in range(400):
    action, _states = dcs_model.predict(obs_fix)
    obac = np.append(obs_raw,action)
    obs_raw, rewards, dones, info = env.step(action)
    inputx = np.append(obac, obs_raw)
    obs_fix = model.predict(inputx)
    sub_reward_fix.append(rewards)

sub_reward_raw=[]
obs = env.reset()
for dd in range(400):
    action, _states = dcs_model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    sub_reward_raw.append(rewards)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(400),np.array(sub_reward_raw))
plt.plot(range(400),np.array(sub_reward_fix))
plt.show()
